In [60]:
import pandas as pd
import pyodbc
import os
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

from datetime import datetime
import numpy as np

In [61]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:/Users/Administrator/LIS/Dabur/python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

# Open Sheet
sheet = client.open_by_key("17NjdYnTDDdj8ssSLP9L06XuXlCEsQDIiIa6rHWICd00") # Master Sheets ( Dimensions Sheet)
Data = sheet.worksheet("Data")
UserSheet = sheet.worksheet("User")

In [62]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

In [63]:
query = """ 
WITH CTE AS 
(SELECT 
(TXT.UserName+'-'+ TXT.VISIT_DATE)AS UserVisitKey ,
TXT.Visit_Date,
TXT.UserName,
(TXT.VISIT_DATE +' '+TXT.CHEKINTIME) AS ChekinTime,
TXT.LISStoreCode,
(TXT.VISIT_DATE+'-'+TXT.LISStoreCode+'-'+TXT.UserName) AS StoreDateKey,
TXT.StoreName,TXT.Designation,
(TXT.VISIT_DATE +' '+TXT.CHECKOUTTIME ) AS Chekouttime,
TXT.FullName,
NoPermissionReason
FROM
(SELECT   FORMAT(CONVERT(DATETIME, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
SA.LISStoreCode,SA.StoreName, SA.UserName,
SA.Designation,SA.FullName,
CONVERT(varchar, CAST([CheckIn-Time] AS TIME), 8) AS CHEKINTIME,
CONVERT(varchar, CAST([CheckOut-Time] AS TIME), 8) AS CHECKOUTTIME,
NoPermissionReason
FROM [PowerBI_LISV2].[GCPLFoodsManagement].[ISPStoreAttendance_NormalExport] AS SA
LEFT JOIN [PowerBI_LISV2].[GCPLFoodsManagement].UserMaster AS UM ON UM.UserName = SA.UserName
WHERE  [Month] = MONTH(GETDATE()-1) AND [Year] = YEAR(GETDATE()-1)
AND UM.Status =  'True' and  SA.UserName not like '%Star%' and SA.UserName not like '%TEST%'
)AS TXT
)
SELECT * FROM CTE;
"""

UserDataQuery ="""
WITH CTE AS
(    SELECT 
        UM2.UserMasterId,
		UM2.EmployeeID,
        UM2.UserName,
        UM2.Status,
		UM2.FullName,
		UM2.Designation,
        UM2.LastWorkingDate,
        UM2.Region,
		UM2.State,
        UM2.City,
        RANK() OVER (PARTITION BY UM2.UserName ORDER BY UM2.UserMasterId DESC) AS RK
    FROM [PowerBI_LISV2].[GCPLFoodsManagement].UserMaster AS UM2
    -- WHERE UM2.UserName IN ('GCPLF-ISP-136')
	)
SELECT CTE.EmployeeID,CTE.UserName,CTE.Status,CTE.FullName,CTE.Designation,CTE.Region,CTE.State,CTE.City,
SUP.Supervisor
FROM CTE
LEFT JOIN
	(SELECT ISP,Supervisor FROM
		(SELECT *,RANK() OVER (PARTITION BY UH.ISP ORDER BY UH.UserHierarchyListId DESC) AS RK FROM
			[PowerBI_LISV2].[GCPLFoodsManagement].UserHierarchyListReport AS UH)AS UH WHERE UH.RK =1) AS SUP
			ON SUP.ISP = CTE.UserName
WHERE CTE.RK = 1 AND CTE.Status = 'True' and 
(CTE.UserName like '%ISP%' OR CTE.UserName like '%MR%' OR CTE.UserName like '%SUP%');

""" 


In [64]:
conn = pyodbc.connect(conn_str)
df = pd.read_sql(query, conn)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_30536\3989963740.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [65]:
UserData = pd.read_sql(UserDataQuery, conn)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_30536\2801009922.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  UserData = pd.read_sql(UserDataQuery, conn)


In [66]:
# UserData['Supervisor'].count()
UserData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   EmployeeID   131 non-null    object
 1   UserName     131 non-null    object
 2   Status       131 non-null    object
 3   FullName     131 non-null    object
 4   Designation  131 non-null    object
 5   Region       131 non-null    object
 6   State        131 non-null    object
 7   City         131 non-null    object
 8   Supervisor   126 non-null    object
dtypes: object(9)
memory usage: 9.3+ KB


In [67]:
Supervisor = UserData[UserData['Designation']=='Supervisor']
Supervisor2 = Supervisor[['UserName','FullName']]
Supervisor2 = Supervisor2.rename(columns={'UserName':'Supervisor','FullName':'SupervisorName'})
Supervisor2

Supervisor  SupervisorName
120  GCPLF-SUP-01  Jitender Kumar
121  GCPLF-SUP-03  Shantanu Barik
122  GCPLF-SUP-05    Sumitha Rani
123  GCPLF-SUP-06      Ravi Kumar

In [68]:
dfUser = pd.merge(UserData,Supervisor2,left_on = 'Supervisor', right_on= 'Supervisor', how='left')

In [69]:
dfUser.head()

EmployeeID      UserName Status             FullName Designation Region  \
0  YM2025009501  GCPLF-ISP-01   True  Suryakanta Pramanik    Promoter   East   
1  YM2025009165  GCPLF-ISP-02   True                GOPAL    Promoter   West   
2  YM2025008537  GCPLF-ISP-03   True          Pampi Barua    Promoter   East   
3  YM2025009264  GCPLF-ISP-04   True       Suparna Mondal    Promoter   East   
4  YM2025010422  GCPLF-ISP-05   True            Sukla Roy    Promoter   East   

          State     City    Supervisor  SupervisorName  
0   WEST BENGAL  Kolkata  GCPLF-SUP-03  Shantanu Barik  
1  Chhattisgarh   Bhilai  GCPLF-SUP-06      Ravi Kumar  
2   WEST BENGAL  Kolkata  GCPLF-SUP-03  Shantanu Barik  
3   WEST BENGAL  Kolkata  GCPLF-SUP-03  Shantanu Barik  
4   WEST BENGAL  Kolkata  GCPLF-SUP-03  Shantanu Barik

In [70]:
UserData2=dfUser.drop(columns = ['FullName','Designation'])

In [71]:

# dfX= df[df['UserName']=='GCPLF-SUP-01']

In [72]:
# dfX.head(2)

In [73]:
# UserData = pd.read_sql(UserDataQuery, conn)
# UserDataX =UserData[UserData['UserName']=='GCPLF-SUP-02']
# UserDataX

In [74]:
print("data extracted....")
loginlogout = pd.merge(df,UserData2, how = 'left',on = 'UserName')

data extracted....


In [75]:
loginlogout['ChekinRank'] = loginlogout.groupby(['StoreDateKey'])['ChekinTime'].rank(method='dense', ascending=True).astype(int)
loginlogout2= loginlogout[loginlogout['ChekinRank']==1]

loginlogout2=loginlogout2.drop(columns = ['ChekinRank'])

loginlogout2 = loginlogout2.drop_duplicates()
loginlogout2.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [76]:
loginlogout2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2506 entries, 0 to 122193
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   UserVisitKey        2506 non-null   object
 1   Visit_Date          2506 non-null   object
 2   UserName            2506 non-null   object
 3   ChekinTime          2506 non-null   object
 4   LISStoreCode        2506 non-null   object
 5   StoreDateKey        2506 non-null   object
 6   StoreName           2506 non-null   object
 7   Designation         2506 non-null   object
 8   Chekouttime         2506 non-null   object
 9   FullName            2506 non-null   object
 10  NoPermissionReason  30 non-null     object
 11  EmployeeID          2495 non-null   object
 12  Status              2495 non-null   object
 13  Region              2495 non-null   object
 14  State               2495 non-null   object
 15  City                2495 non-null   object
 16  Supervisor          2345 no

In [77]:
loginlogout2= loginlogout2[['UserVisitKey','Visit_Date','UserName','ChekinTime','LISStoreCode','StoreDateKey','StoreName','Designation',
                           'Chekouttime','FullName','NoPermissionReason','EmployeeID',
                           'Region','State','City','Status','Supervisor','SupervisorName','LastUpdated']]